This python file contains the functions for our final project.

In [2]:
import pandas as pd
import kagglehub
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from shapely.ops import transform
import pyproj
import matplotlib.pyplot as plt

In [3]:
def load_philly_crime_data(years, base_path="data", filename_template="philadelphia_crime_data_{}.csv"):
    """
    Loads and concatenates Philadelphia crime data across multiple years.

    Parameters:
        years (list): List of years to load (e.g. [2021, 2022, 2023])
        base_path (str): Folder where files are stored
        filename_template (str): Filename format with one {} placeholder for the year

    Returns:
        pd.DataFrame: Combined DataFrame of crime data with 'date' and 'year' columns
    """
    crime_dfs = []

    for year in years:
        file_path = f"{base_path}/{filename_template.format(year)}"
        df = pd.read_csv(file_path)
        df["year"] = year
        df["dispatch_date_time"] = pd.to_datetime(df["dispatch_date_time"])
        df["date"] = df["dispatch_date_time"].dt.floor("D")
        crime_dfs.append(df)

    return pd.concat(crime_dfs, ignore_index=True)


In [4]:
def nfl_game_date(year, team):
    # load the specific year
    path_nfl = kagglehub.dataset_download("keonim/nfl-game-scores-dataset-2017-2023")
    df = pd.read_csv(f"{path_nfl}/Season_Scores/{year}_scores.csv")
    df = df.dropna(subset=['Date'])

    # subsets the df to games where the team is playing
    df_away = df[(df["AwayTeam"] == team)]
    df_home = df[(df["HomeTeam"] == team)]

    home_dates_with_year = df_home['Date'].apply(
        lambda x: f"{x}/{year+1}" if int(str(x).split('/')[0]) <= 6 
        else f"{x}/{year}")    
    away_dates_with_year = df_away['Date'].apply(
        lambda x: f"{x}/{year+1}" if int(str(x).split('/')[0]) <= 6 
        else f"{x}/{year}")

    # Now convert to datetime
    home_game_dates = pd.to_datetime(home_dates_with_year)
    away_game_dates = pd.to_datetime(away_dates_with_year)

    # Gets the result of the game
    home_game_win = df_home['HomeWin'].to_list()
    away_game_win = df_away['AwayWin'].to_list()

    # Convert to "Win" or "Loss"
    home_game_win = ["Win" if bool(x) else "Loss" for x in home_game_win]
    away_game_win = ["Win" if bool(x) else "Loss" for x in away_game_win]

    # Creates an away, home df, with the pairs for results and tags the Location
    home = pd.DataFrame(list(zip(home_game_dates, home_game_win)))
    away = pd.DataFrame(list(zip(away_game_dates, away_game_win)))
    home["Location"] = "Home"
    away["Location"] = "Away"

    # Puts the two together and sorts them by date
    season = pd.concat([home, away], ignore_index=True) 
    season.columns = ["Date", "Result", "Location"]
    season = season.sort_values('Date').reset_index(drop=True)    

    return season

In [5]:
def nba_game_date(team,year):
    #first get teamId for this dataset
    team_df = pd.read_csv(f"{path_nba}/TeamHistories.csv")

    #searches the dataset for the ID, returns nothing if not found
    team_id = team_df[team_df['teamName'] == team]['teamId']
    if not team_id.empty:
        team_id = team_id.iloc[0]
    else:
        print("Team not found in dataset, look at the documentation.")
        return

    #use team_id and year to sort games wanted
    df = pd.read_csv(f"{path_nba}/Games.csv", low_memory=False)
    df["gameDate"] = pd.to_datetime(df["gameDate"])
    start = pd.to_datetime(f"{year}-10-01")
    end = pd.to_datetime(f"{year+1}-06-30")
    df_home = df[(df["hometeamId"] == team_id) & ((df['gameDate'] >= start) & (df['gameDate'] <= end))]
    df_away = df[(df["awayteamId"] == team_id) & ((df['gameDate'] >= start) & (df['gameDate'] <= end))]


    home_game_dates = df_home["gameDate"].to_list()
    away_game_dates = df_away["gameDate"].to_list()
    
    home_game_win = [x == team_id for x in df_home['winner']]
    away_game_win = [x == team_id for x in df_away['winner']]

    #Creates an away, home df, with the pairs for results and tags the Location
    home = pd.DataFrame(list(zip(home_game_dates, home_game_win)))
    away = pd.DataFrame(list(zip(away_game_dates, away_game_win)))
    home["Location"] = "Home"
    away["Location"] = "Away"
    season = pd.concat([home, away], ignore_index=True) 
    season.columns = ["Date", "Result", "Location"]
    season = season.sort_values('Date').reset_index(drop=True)    
    
    return season

In [6]:
def load_all_eagles_games(start_year=2021, end_year=2024):
    """
    Load all Eagles NFL games across multiple years and add a clean 'date' and 'year' column.

    Parameters:
        start_year (int): Start year (inclusive)
        end_year (int): End year (inclusive)

    Returns:
        pd.DataFrame: Combined DataFrame of all games with 'date', 'year', and 'Location'
    """
    all_games = []
    for year in range(start_year, end_year + 1):
        games = nfl_game_date(year, "Eagles").copy()
        games["date"] = pd.to_datetime(games["Date"]).dt.date
        games["year"] = year
        all_games.append(games)
    return pd.concat(all_games, ignore_index=True)

In [7]:
def load_all_bears_games(start_year=2021, end_year=2024):
    """
    Load all Bears NFL games across multiple years and add a clean 'date' and 'year' column.

    Parameters:
        start_year (int): Start year (inclusive)
        end_year (int): End year (inclusive)

    Returns:
        pd.DataFrame: Combined DataFrame of all games with 'date', 'year', and 'Location'
    """
    all_games = []
    for year in range(start_year, end_year + 1):
        games = nfl_game_date(year, "Bears").copy()
        games["date"] = pd.to_datetime(games["Date"]).dt.date
        games["year"] = year
        all_games.append(games)
    return pd.concat(all_games, ignore_index=True)


In [8]:
def preprocess_crime_data(df, stadium_coords, date_col,zip_shapes=None,lat_col='lat',lng_col='lng'):
    """
    Preprocess crime data:
    - Parses date column
    - Converts to GeoDataFrame using lat/lng
    - Computes distance to stadium
    - Optionally joins ZIP code geometries
    
    Parameters:
        df (pd.DataFrame): Raw crime data with 'lat', 'lng', 'date' columns
        stadium_coords (tuple): (longitude, latitude) of stadium
        zip_shapes (gpd.GeoDataFrame, optional): ZIP code shapefile with 'geometry' and 'zip_code'
    
    Returns:
        gpd.GeoDataFrame: Crime GeoDataFrame with distance and optional ZIP code
    """
    # Clean and parse
    df = df.copy()
    df['date'] = pd.to_datetime(df[date_col], errors='coerce')
    df['lat'] = pd.to_numeric(df[lat_col], errors='coerce')
    df['lng'] = pd.to_numeric(df[lng_col], errors='coerce')
    df = df.dropna(subset=['lat', 'lng', 'date'])

    # Geo conversion
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lng'], df['lat']), crs="EPSG:4326")
    gdf = gdf.to_crs(epsg=3857)

    # Project stadium point
    stadium_point = Point(stadium_coords)
    stadium_proj = gpd.GeoSeries([stadium_point], crs="EPSG:4326").to_crs(epsg=3857).iloc[0]

    # Distance in meters
    gdf['distance_to_stadium_m'] = gdf.geometry.distance(stadium_proj)

    # Optional ZIP join
    if zip_shapes is not None:
        if 'zip' in zip_shapes.columns:
            zip_shapes = zip_shapes.rename(columns={"zip": "zip_code"})
        gdf = gpd.sjoin(gdf, zip_shapes[['geometry', 'zip_code']], how="left", predicate="within")

    return gdf



In [9]:
def tag_game_windows(df, games_df, team_name=None, window_hours=6, date_col='date'):
    """
    Tag crimes that occur during a time window after a game.

    Parameters:
        df (pd.DataFrame): Crime data with a datetime column (default 'date').
        games_df (pd.DataFrame): Game schedule with 'Date' column (datetime).
        team_name (str, optional): For labeling only.
        window_hours (int): Hours after game start to include in the window.
        date_col (str): Column in df to compare with game times.

    Returns:
        pd.DataFrame: Original crime data with 'is_game_window' column added.
    """
    df = df.copy()
    games_df = games_df.copy()

    # Parse and clean timestamps in crime data
    if date_col not in df.columns:
        raise ValueError(f"'{date_col}' not found in DataFrame columns.")

    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    df[date_col] = df[date_col].dt.tz_localize(None)
    df = df.dropna(subset=[date_col])

    # Parse and clean game dates
    games_df['Date'] = pd.to_datetime(games_df['Date'], errors='coerce').dt.tz_localize(None)
    games_df = games_df.dropna(subset=['Date'])

    # Build game windows (start → start + X hours)
    game_windows = [(start, start + pd.Timedelta(hours=window_hours)) for start in games_df['Date']]

    # Check if each timestamp falls within any game window
    def in_any_window(ts):
        return any(start <= ts <= end for start, end in game_windows)

    df['is_game_window'] = df[date_col].apply(in_any_window)

    return df


In [10]:
def compute_distance_crime_delta(crime_df, distance_col='distance_to_stadium_m', 
                                  is_game_col='is_game_window', bin_size_m=1000, max_dist_m=5000):
    """
    Compute change in average crime counts by distance band from the stadium.

    Parameters:
        crime_df (pd.DataFrame): Crime data with distance and game window indicators.
        distance_col (str): Column name for distance from stadium in meters.
        is_game_col (str): Boolean column indicating game-related crime windows.
        bin_size_m (int): Width of distance bins in meters.
        max_dist_m (int): Maximum distance to consider (bins go from 0 to this value).

    Returns:
        pd.DataFrame: Δ crime statistics per distance band.
    """
    df = crime_df.copy()

    # Filter out null distances
    df = df[df[distance_col].notnull()]

    # Create distance bins
    bins = np.arange(0, max_dist_m + bin_size_m, bin_size_m)
    labels = [f"{int(low/1000)}–{int(high/1000)}km" for low, high in zip(bins[:-1], bins[1:])]
    df["distance_band_km"] = pd.cut(df[distance_col], bins=bins, labels=labels, include_lowest=True)

    # Group by game/non-game and distance band
    grouped = df.groupby([is_game_col, "distance_band_km"]).size().reset_index(name="crime_count")

    # Pivot to get side-by-side counts
    pivot = grouped.pivot(index="distance_band_km", columns=is_game_col, values="crime_count").fillna(0)
    pivot.columns = ['non_game_crime', 'game_crime'] if False in pivot.columns else ['game_crime', 'non_game_crime']
    
    # Normalize by number of days in each category
    n_game_days = df[df[is_game_col]].date.dt.date.nunique()
    n_non_game_days = df[~df[is_game_col]].date.dt.date.nunique()

    pivot["avg_game_crime"] = pivot["game_crime"] / n_game_days
    pivot["avg_non_game_crime"] = pivot["non_game_crime"] / n_non_game_days
    pivot["delta_crime"] = pivot["avg_game_crime"] - pivot["avg_non_game_crime"]

    return pivot.reset_index()



In [11]:
def filter_sundays(df, datetime_col="dispatch_date_time"):
    """
    Filter a DataFrame to include only records from Sundays.

    Parameters:
        df (pd.DataFrame): Crime data including a datetime column.
        datetime_col (str): Name of the datetime column to use (default is 'dispatch_date_time').

    Returns:
        pd.DataFrame: Filtered DataFrame containing only Sunday records.
    """
    df = df.copy()
    df[datetime_col] = pd.to_datetime(df[datetime_col], errors='coerce')
    return df[df[datetime_col].dt.dayofweek == 6]  # 6 = Sunday


In [12]:
def plot_stadium_radii_map_with_bar_chart(
    stadium_coords,
    crime_df,
    radii_meters=[1000, 2000, 3000],
    zoom=15,
    title="Stadium Radius Map + Δ Crime by Distance Band",
    lat_col='lat',
    lng_col='lng'
):
    """
    Plot a map with concentric radius bands around a stadium and show bar chart of crime counts in each band.

    Parameters:
        stadium_coords (tuple): (longitude, latitude) of the stadium
        crime_df (pd.DataFrame): Must include 'distance_to_stadium_m' and 'is_game_window' columns
        radii_meters (list): List of radius values in meters
        zoom (int): Zoom level for the map
        title (str): Title of the map
    """
    # Set up projection
    stadium_lon, stadium_lat = stadium_coords
    proj = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True).transform
    inverse_proj = pyproj.Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True).transform
    point_proj = transform(proj, Point(stadium_lon, stadium_lat))

    # Create GeoDataFrame bands (excluding outermost "catch-all" which has no boundary)
    bands = []
    for r in radii_meters:
        buffer = transform(inverse_proj, point_proj.buffer(r))
        bands.append(gpd.GeoDataFrame({'radius_m': [r]}, geometry=[buffer], crs="EPSG:4326"))

    # Build map
    fig_map = go.Figure()
    fig_map.update_layout(
        mapbox_style="carto-positron",
        mapbox_zoom=zoom,
        mapbox_center={"lat": stadium_lat, "lon": stadium_lon},
        title=title
    )

    for gdf in bands:
        geojson = gdf.__geo_interface__
        radius = gdf['radius_m'].iloc[0]
        fig_map.add_trace(go.Choroplethmapbox(
            geojson=geojson,
            locations=[0],
            z=[radius],
            colorscale='Blues',
            showscale=False,
            name=f"{radius/1000:.1f} km radius",
            marker_opacity=0.3,
            marker_line_width=1
        ))

    fig_map.add_trace(go.Scattermapbox(
        lat=[stadium_lat],
        lon=[stadium_lon],
        mode='markers+text',
        marker=go.scattermapbox.Marker(size=10, color='red'),
        text=["Stadium"],
        textposition="top right"
    ))

    # Categorize crimes into bands including an outer "catch-all" band
    bins = [0] + radii_meters + [np.inf]
    labels = [f"{bins[i]/1000:.1f}–{bins[i+1]/1000:.1f} km" for i in range(len(bins)-2)]
    labels.append(f">{radii_meters[-1]/1000:.1f} km")
    crime_df["band"] = pd.cut(crime_df["distance_to_stadium_m"], bins=bins, labels=labels)

    # Count number of game and non-game days
    num_game_days = crime_df["is_game_window"].sum()
    num_non_game_days = (~crime_df["is_game_window"]).sum()

    if num_game_days == 0 or num_non_game_days == 0:
        raise ValueError("Game or non-game days missing in data.")

    # Group by band and game window
    band_counts = crime_df.groupby(["band", "is_game_window"]).size().unstack(fill_value=0)
    band_counts["avg_game"] = band_counts[True] / num_game_days
    band_counts["avg_non_game"] = band_counts[False] / num_non_game_days
    band_counts["Δ crime"] = band_counts["avg_game"] - band_counts["avg_non_game"]
    band_counts = band_counts.reset_index()

    # Plot bar chart
    fig_bar = px.bar(
        band_counts,
        x="band",
        y="Δ crime",
        title="Δ Crime per Day by Distance Band",
        labels={"band": "Distance Band", "Δ crime": "Δ Crimes per Day (Game - Non-game)"},
        text_auto=True
    )

    fig_map.show()
    fig_bar.show()
